# <font color='black'>Регрессионный анализ: продолжение, 2024 </font>
## <font color='black'>Практическое занятие 1 </font>
В рамках данного практического занятия мы обратимся к данным из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Возьмем такой массив, так как мы пока не знакомились с моделями для анализа панельных данных.

Стоит отметить, что авторы изучают взаимосвязь уровня коррупции (является откликом в регрессионной модели) и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Приведен к непрерывной шкале от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения: World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import seaborn as sns
import numpy as np

Откроем массив данных для репликации результатов исследования: lab1.dta

In [ ]:
lab1 = pd.read_stata('lab1.dta')

Познакомимся с тем, как устроен массив данных.

In [ ]:
lab1.head(10)

Ниже представим описательные статистики.

In [ ]:
lab1.describe()

Построим диаграмму рассеяния между зависимой переменной (cpi) и ключевым предиктором - уровнем демократии (dem). Что можно сказать про характер связи между коррупцией и демократией?

In [ ]:
sns.scatterplot(data = lab1, x = "dem", y = "cpi")

Оценим первую регрессионную модель (m1). Это будет парная линейная регрессия: пока учитываем только ключевой предиктор - уровень демократии, контрольные переменные, предложенные авторами статьи, включим на следующем практическом занятии.

Проинтерпретируйте полученные результаты.

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem", data = lab1).fit()
print(m1.summary())

Отдельно можно вывести вектор оценок параметров модели. Если хотите вывести оценку определенного параметра, то в квадратных скобках укажите его номер, не забываем, что в Python отсчет идет с 0, поэтому константе соответствует 0, а не 1.

In [ ]:
m1.params.iloc[1]

Также можно вывести ковариационную матрицу оценок параметров. Какая информация в ней содержится? По главной диагонали - дисперсии оценок параметров, остальные элементы - ковариации между оценками коэффициентов (совместная изменчивость). Если взять квадратный корень элементов по главной диагонали, то получим стандартные ошибки оценок коэффициентов, которые нам пригодятся при проверке гипотезы о незначимости коэффициентов в регрессионной модели.    

In [ ]:
m1.cov_params()

Представим полученный объект как матрицу в Python, чтобы можно было обратиться к определенным элементам. И давайте посмотрим, как рассчитать p-value в случае проверки гипотезы о незначимости коэффициента при предикторе dem против двусторонней альтернативы.

Статистика критерия рассчитывается следующим образом: $\dfrac{\hat{b}}{se}$. При верной нулевой гипотезе статистика имеет распределение Стьюдента с количеством степеней свободы равным $n-k=1$, где $k$ - количество предикторов в модели, не забываем про еще один параметр - это константа.

Сделайте вывод на основе полученного p-value.

In [ ]:
Cov_m1 = np.asmatrix(m1.cov_params())
stats.t.sf(abs(m1.params.iloc[1]/np.sqrt(Cov_m1[1,1])), m1.nobs - m1.df_model - 1)*2

Для того же коэффициента построим 95%-ый доверительный интервал. В качестве отправной точки используем оценку соответствующего коэффициента $\hat{\beta}$ при dem. Также для того, чтобы задать границы, нам понадобится стандартная ошибка оценки коэффициента и критическая точка (квантиль по распределению Стьюдента уровня 0.975, $df = n - k - 1$).

Проинтерпретируйте полученный доверительный интервал.

In [ ]:
left_boundary = m1.params.iloc[1] - np.sqrt(Cov_m1[1,1])*stats.t.ppf(0.975, m1.nobs - m1.df_model - 1)
right_boundary = m1.params.iloc[1] + np.sqrt(Cov_m1[1,1])*stats.t.ppf(0.975, m1.nobs - m1.df_model - 1)

print(left_boundary.round(4), right_boundary.round(4))

Выведем также ANOVA-таблицу по результатам оцененной модели m1:

In [ ]:
anova = sm.stats.anova_lm(m1)
print(anova)